# Create datasets for congressional CEJA stances

In [ ]:
# imports
from pandasql import sqldf
import pandas as pd
from fuzzywuzzy import fuzz

## Ceja Sponsorship

In [ ]:
# load data
ceja_supporters_reps = pd.read_csv('data/government/illinois/legislature/ceja/ceja-sponsors-reps.tsv', sep='\t', comment="#")
ceja_supporters_sens = pd.read_csv('data/government/illinois/legislature/ceja/ceja-sponsors-sens.tsv', sep='\t', comment="#")

reps = pd.read_csv('data/government/illinois/legislature/reps-102.tsv', sep='\t', comment="#")
sens = pd.read_csv('data/government/illinois/legislature/sens-102.tsv', sep='\t', comment="#")

In [ ]:
len(ceja_supporters_sens)

In [ ]:
# use fuzzy strings to match politician names
# write the version in the rep/sen name CSV as a source of truth
# https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe
# TODO sen join seems wrong? there's 34 sens in the list of names of sponsors, but less than that in the join
# maybe some sponsors are no longer in office, so not in that list of sens?

for rep in reps['name']:
    for i in range(len(ceja_supporters_reps.index)):
        if fuzz.partial_ratio(rep, ceja_supporters_reps['name'][i]) > 90:
            print(f"matched {rep} to {ceja_supporters_reps['name'][i]}")
            ceja_supporters_reps['name'][i] = rep
            break
                  
for sen in sens['name']:
    for i in range(len(ceja_supporters_sens.index)):
        if fuzz.partial_ratio(sen, ceja_supporters_sens['name'][i]) > 90:
            print(f"matched {sen} to {ceja_supporters_sens['name'][i]}")
            ceja_supporters_sens['name'][i] = sen
            break

In [ ]:
q_reps = """
SELECT reps.name, reps.district, 'True' as supports_ceja
FROM reps, ceja_supporters_reps
WHERE reps.name = ceja_supporters_reps.name
UNION
SELECT reps.name, reps.district, 'False' as supports_ceja
FROM reps
WHERE NOT EXISTS (
    SELECT NULL
    FROM ceja_supporters_reps
    WHERE reps.name = ceja_supporters_reps.name
)
"""
ceja_support_reps = sqldf(q_reps, globals())

In [ ]:
q_sens = """
SELECT sens.name, sens.district, 'True' as supports_ceja
FROM sens, ceja_supporters_sens
WHERE sens.name = ceja_supporters_sens.name
UNION
SELECT sens.name, sens.district, 'False' as supports_ceja
FROM sens
WHERE NOT EXISTS (
    SELECT NULL
    FROM ceja_supporters_sens
    WHERE sens.name = ceja_supporters_sens.name
)
"""
ceja_support_sens = sqldf(q_sens, globals())

In [ ]:
len(ceja_support_sens[ceja_support_sens['supports_ceja']=="False"])

In [ ]:
ceja_support_reps.to_csv('data/government/illinois/legislature/ceja/ceja-sponsorship-reps.tsv', sep="\t", index=False)
ceja_support_sens.to_csv('data/government/illinois/legislature/ceja/ceja-sponsorship-sens.tsv', sep="\t", index=False)

## Processing Data for SB2408 Votes

In [95]:
# load data
ceja_votes_reps = pd.read_csv('data/government/illinois/legislature/ceja/sb2408-votes-house-raw.tsv', sep='\t', comment="#")
ceja_votes_sens = pd.read_csv('data/government/illinois/legislature/ceja/sb2408-votes-senate-raw.tsv', sep='\t', comment="#")

reps = pd.read_csv('data/government/illinois/legislature/reps-102.tsv', sep='\t', comment="#")
sens = pd.read_csv('data/government/illinois/legislature/sens-102.tsv', sep='\t', comment="#")

In [97]:
# use fuzzy strings to match politician names
# NOTE: the matching here doesn't catch every match - between this cell and the next, I manually finish matching the names in the CSV
# write the version in the rep/sen name CSV as a source of truth
# https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe

ceja_votes_reps['name'] = None
ceja_votes_sens['name'] = None

for rep in reps['name']:
    for i in range(len(ceja_votes_reps.index)):
        parsed_name = ceja_votes_reps['name_raw'][i][5:].split('[')[0]
        ln, fn = parsed_name.split(', ')
        rep_name = f"{fn} {ln}"

        if fuzz.partial_ratio(rep, rep_name) > 90:
            ceja_votes_reps['name'][i] = rep
            break
                  
for sen in sens['name']:
    for i in range(len(ceja_votes_sens.index)):
        parsed_name = ceja_votes_sens['name_raw'][i][5:].split('[')[0]
        ln, fn = parsed_name.split(', ')
        sen_name = f"{fn} {ln}"

        if fuzz.partial_ratio(sen, sen_name) > 90:
            ceja_votes_sens['name'][i] = sen
            break

ceja_votes_reps.to_csv('data/government/illinois/legislature/ceja/sb2408-votes-house.tsv', sep="\t", index=False)
ceja_votes_sens.to_csv('data/government/illinois/legislature/ceja/sb2408-votes-senate.tsv', sep="\t", index=False)

In [105]:
# NOTE: not all names were matched by fuzzy matching, I will manually join the remaining name entries before performing this join
# this might be sacrilege, I'll note it here -> John D'Amico resigned, I'll fill his parsed "name" entry with the person now in his seat (Michael Kelly)
ceja_votes_reps = pd.read_csv('data/government/illinois/legislature/ceja/sb2408-votes-house.tsv', sep='\t', comment="#")
ceja_votes_sens = pd.read_csv('data/government/illinois/legislature/ceja/sb2408-votes-senate.tsv', sep='\t', comment="#")

if any(ceja_votes_reps['name'].isnull()):
    print(ceja_votes_reps[ceja_votes_reps['name'].isnull()])
    raise Exception("not all names matched)")
else:
    reps_district_join = reps[['name', 'district']]
    ceja_votes_reps = ceja_votes_reps.merge(reps_district_join, on='name', how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)').astype({'district': 'int32'})

if any(ceja_votes_sens['name'].isnull()):
    print(ceja_votes_reps[ceja_votes_sens['name'].isnull()])
    raise Exception("not all names matched)")
else:
    sens_district_join = sens[['name', 'district']]
    ceja_votes_sens = ceja_votes_sens.merge(sens_district_join, on='name', how='left', suffixes=('', '_DROP')).filter(regex='^(?!.*_DROP)').astype({'district': 'int32'})

ceja_votes_reps.to_csv('data/government/illinois/legislature/ceja/sb2408-votes-house.tsv', sep="\t", index=False)
ceja_votes_sens.to_csv('data/government/illinois/legislature/ceja/sb2408-votes-senate.tsv', sep="\t", index=False)

# Processing coordinate data of Energy Plants

In [ ]:
import re

def convertDMSToDD(degrees: str, minutes: str, seconds: str, direction: str):
	dd = float(degrees) + (float(minutes)/60) + (float(seconds)/(60*60))

	if direction == "S" or direction == "W":
		dd *= -1

	return dd

def castDMS(input: str):
	"""
	cast input from coord to decimal

	old regex: /\d+(\.\d+)?°(\d+(\.\d+)?′)?(\d+(\.\d+)?″)?[NESW]/
	"""
	parts = re.split("[^\d\w\.]+", input)

	if re.match("\d+(\.\d+)?°\d+(\.\d+)?′\d+(\.\d+)?″[NESW]", input):
		# 38°16′40.2″N
		return convertDMSToDD(parts[0], parts[1], parts[2], parts[3])
	elif re.match("\d+(\.\d+)?°\d+(\.\d+)?′[NESW]", input):
		# 38.204°N
		return convertDMSToDD(parts[0], parts[1], 0, parts[2])
	elif re.match("\d+(\.\d+)?°[NESW]", input):
		# 38°16.12N
		return convertDMSToDD(parts[0], 0, 0, parts[1])
	else:
		# it's something else, just return original input
		return input

In [ ]:
import pandas as pd

csv_path = "data/energy/illinois/natural_gas_plants.tsv"
df = pd.read_csv(csv_path, delimiter="\t")

if 'coordinates' in df:
    df['yCoordinate'] = df.apply(lambda row: castDMS(row.coordinates.split(' ')[0]), axis=1)
    df['xCoordinate'] = df.apply(lambda row: castDMS(row.coordinates.split(' ')[1]), axis=1)
    df.to_csv(csv_path, sep="\t", index=False)
else:
    print("'coordinates' column not found in the file")